In [1]:
import numpy as np
import PIL
import umap
import pandas as pd
import json
import glob
from pytorch_metric_learning import distances, losses, miners, reducers
import torch.nn as nn
import os
import matplotlib.pyplot as plt

In [2]:
import torch
import torchvision
from torchvision import models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
from pytorch_metric_learning import losses, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.distances import LpDistance,CosineSimilarity
from pytorch_metric_learning.utils.inference import CustomKNN

In [3]:
%matplotlib inline
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms
import torch.optim as optim
import time
import tqdm as tqdm
from torch.autograd import Variable

In [ ]:
import wandb
import random  # for demo script

wandb.login()

In [5]:
config = {
    'name':'cub_triplet_loss_epshn_resnet50_sgd_aug',
    'dataset':'CUB_200_2011',
    'random_seed':42,
    'model_architecture':'resnet50',
    'embedding_dim':128,
    'distance':'cosine',
    'image_height':224,
    'image_width':224,
    'train_test_split':0.2,
    'class_split':0.1,
    'batch_size':128,
    'optimizer':'sgd',
    'learning_rate':0.001,
    'num_epochs':100,
    'loss':'NTXentLoss',
    'miner':'epshn',
    'reducer':0,
    'metric':'precision_at_1',
    'model_save_path':'models/cub_triplet_loss_epshn_resnet50_sgd_aug',
    'temperature': 0.1
}

In [6]:
id = wandb.util.generate_id()
run = wandb.init(
    id=id,
    name = config['name'],
    # Set the project where this run will be logged
    project="embedding_based_classification",
    # Track hyperparameters and run metadata
    config=config,
    resume="allow"
)
print(id)

jxymdca8


In [7]:
# Access the run
#api = wandb.Api()
#run = api.run('pranavjadhav001/embedding_based_classification/0ypuvl4r')

In [8]:
np.random.seed(config['random_seed'])
torch.manual_seed(config['random_seed'])
torch.cuda.manual_seed(config['random_seed'])
torch.backends.cudnn.deterministic = False

In [18]:
# print(models.resnet18())
class ResNetFeatrueExtractor50(nn.Module):
    def __init__(self, pretrained = True):
        super(ResNetFeatrueExtractor50, self).__init__()
        self.model = models.resnet50(pretrained=pretrained)
        self.model.fc = nn.Linear(2048, config['embedding_dim'])
        
    def forward(self, x):
        x = self.model(x)
        return x

In [20]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func,mining_func, device, train_loader, optimizer, epoch):
    model.train()
    train_losses = []
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        train_losses.append(loss)
        if batch_idx % 100 == 0:
            print("Epoch {} Iteration {}: Loss = {}".format(epoch, batch_idx, loss))
    return torch.mean(torch.tensor(train_losses)).item()
    
### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester(dataloader_num_workers=0)
    return tester.get_all_embeddings(dataset, model)

### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, test_labels, train_embeddings, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))
    return accuracies["precision_at_1"]
    
device = torch.device("cuda")

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# create train and test transforms
transform = transforms.Compose(
    [
        transforms.Resize((config['image_height'], config['image_width'])),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

batch_size = 128

In [21]:
def tra_transforms(imgsize, RGBmean, RGBstdv):
    return transforms.Compose([transforms.Resize(int(imgsize*1.1)),
                                 transforms.RandomCrop(imgsize),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize(RGBmean, RGBstdv)])

def eva_transforms(imgsize, RGBmean, RGBstdv):
    return transforms.Compose([transforms.Resize(imgsize),
                                 transforms.CenterCrop(imgsize),
                                 transforms.ToTensor(),
                                 transforms.Normalize(RGBmean, RGBstdv)])


In [22]:
train_transform = tra_transforms(224,mean,std)
test_transform = eva_transforms(224,mean,std)

In [23]:
with open('CUB_200_2011/classes.txt','r') as f:
    classes = f.readlines()
classes = [i.replace('\n','') for i in classes]
classes = [i.split(' ')[1] for i in classes]
class_dict = {k:v for k,v in zip(classes,range(200))}

In [24]:
image_paths = []
labels = []
for folder_path,i in class_dict.items():
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    image_paths.extend(folder_images)
    labels.extend([i]*len(folder_images))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=config['train_test_split'],
                                                    stratify=labels,
                                                    random_state=config['random_seed'])

In [26]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

9430 2358 9430 2358


In [27]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class CUBDataset(Dataset):
    def __init__(self, image_paths,labels,transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.load_image_from_paths()
        
    def load_image_from_paths(self):
        self.images = []
        for i in self.image_paths:
            img = PIL.Image.open(i)
            if len(img.getbands()) ==1 :
                img = img.convert("RGB")
            self.images.append(img)
            
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [31]:
train_dataset  = CUBDataset(X_train,y_train,train_transform)
test_dataset  = CUBDataset(X_test,y_test,test_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [28]:
model = ResNetFeatrueExtractor50()
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'])
num_epochs = config['num_epochs']

### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.MeanReducer()
loss_func = losses.NTXentLoss(temperature=config['temperature'], distance=distance, reducer=reducer)
mining_func = miners.BatchEasyHardMiner(pos_strategy="easy",neg_strategy="semihard")
knn_func = CustomKNN(CosineSimilarity())
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",),k=1,knn_func=knn_func,avg_of_avgs=True,return_per_class=False)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
wandb.watch(model, log='all')

[]

In [32]:
#total_loss = []
#total_acc = []
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, loss_func,mining_func, device, train_loader, optimizer, epoch)
    #total_loss.extend(train_loss)
    test_acc = test(train_dataset, test_dataset, model, accuracy_calculator)
    #total_acc.append(test_acc)
    wandb.log({"test_accuracy": test_acc, "train_loss": train_loss,'epoch':epoch})

Epoch 1 Iteration 0: Loss = 0.6651296615600586


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:11<00:00,  6.65it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.3679785353535354
Epoch 2 Iteration 0: Loss = 0.618576169013977


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:04<00:00, 14.82it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.4312171717171717
Epoch 3 Iteration 0: Loss = 0.5853459239006042


...

Computing accuracy
Test set accuracy (Precision@1) = 0.755743686868687
Epoch 99 Iteration 0: Loss = 0.266751229763031


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:04<00:00, 14.96it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.7604633838383837
Epoch 100 Iteration 0: Loss = 0.24612486362457275


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:04<00:00, 15.05it/s]

Computing accuracy
Test set accuracy (Precision@1) = 0.7615189393939394


In [34]:
wandb.unwatch()

In [35]:
torch.save(model,'models/cub_triplet_loss_epshn_resnet50_sgd_aug_200.pth')

In [19]:
model = torch.load('models/cub_triplet_loss_epshn_resnet18_sgd_aug_200.pth',map_location='cuda')

In [36]:
model.eval()

ResNetFeatrueExtractor50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequent

In [37]:
from pytorch_metric_learning.distances import LpDistance,CosineSimilarity
from pytorch_metric_learning.utils.inference import CustomKNN
knn_func = CustomKNN(CosineSimilarity())
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",),k=1,knn_func=knn_func,avg_of_avgs=False,return_per_class=True)
#test(train_dataset, test_dataset, model, accuracy_calculator)

In [38]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:04<00:00, 14.99it/s]


In [39]:
accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, test_labels, train_embeddings, train_labels)
print(accuracies)

{'precision_at_1': [0.9166666666666666, 0.6666666666666666, 0.8333333333333334, 1.0, 0.7777777777777778, 0.875, 1.0, 0.6666666666666666, 0.4166666666666667, 1.0, 0.5833333333333334, 1.0, 0.8333333333333334, 0.8333333333333334, 0.8181818181818182, 0.7272727272727273, 0.9090909090909091, 1.0, 0.9166666666666666, 0.8333333333333334, 0.9166666666666666, 0.8181818181818182, 0.4166666666666667, 1.0, 0.6666666666666666, 0.75, 0.3333333333333333, 1.0, 0.4166666666666667, 0.08333333333333333, 0.5833333333333334, 0.6, 0.5833333333333334, 1.0, 0.9166666666666666, 0.75, 0.6666666666666666, 0.8333333333333334, 0.5833333333333334, 0.5, 0.9166666666666666, 1.0, 0.25, 0.6666666666666666, 0.75, 1.0, 0.9166666666666666, 1.0, 0.75, 0.8333333333333334, 0.6666666666666666, 0.8333333333333334, 0.9166666666666666, 0.75, 1.0, 1.0, 0.9166666666666666, 0.8333333333333334, 0.16666666666666666, 0.3333333333333333, 0.9166666666666666, 0.25, 0.9166666666666666, 0.4166666666666667, 0.6, 0.6666666666666666, 0.4166666

In [40]:
data_dict = {v:k for k,v in class_dict.items()}

In [41]:
acc_dict = {}
for i in test_labels.unique():
    new_labels = test_labels[test_labels==i]
    new_embeddings = test_embeddings[test_labels==i]
    accuracies = accuracy_calculator.get_accuracy(
        new_embeddings, new_labels, train_embeddings, train_labels, False
    )
    acc_dict[data_dict[int(i.detach().cpu().numpy())]]=[ 
                  len(new_labels),
                  len(train_labels[train_labels==i]),
                  accuracies["precision_at_1"][0]]
    print("{:<30} test samples {:<5}, training samples {:<5}: {}".format(data_dict[int(i.detach().cpu().numpy())],
                                                                  len(new_labels),
                                                                  len(train_labels[train_labels==i]),
                                                                  accuracies["precision_at_1"]))

001.Black_footed_Albatross     test samples 12   , training samples 48   : [0.9166666666666666]
002.Laysan_Albatross           test samples 12   , training samples 48   : [0.6666666666666666]
003.Sooty_Albatross            test samples 12   , training samples 46   : [0.8333333333333334]
004.Groove_billed_Ani          test samples 12   , training samples 48   : [1.0]
005.Crested_Auklet             test samples 9    , training samples 35   : [0.7777777777777778]
006.Least_Auklet               test samples 8    , training samples 33   : [0.875]
007.Parakeet_Auklet            test samples 10   , training samples 43   : [1.0]
008.Rhinoceros_Auklet          test samples 9    , training samples 39   : [0.6666666666666666]
009.Brewer_Blackbird           test samples 12   , training samples 47   : [0.4166666666666667]
010.Red_winged_Blackbird       test samples 12   , training samples 48   : [1.0]
011.Rusty_Blackbird            test samples 12   , training samples 48   : [0.5833333333333334]
01

In [42]:
all_table = [[k]+v for k,v in acc_dict.items()]

In [43]:
columns = ["class_name", "no. of test samples", "no. of train samples", "precision@1"]
train_table = wandb.Table(data=all_table, columns=columns)

In [44]:
run.log({"all_classes_metrics": train_table})

In [45]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:04<00:00, 15.00it/s]


In [47]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:05<00:00, 13.62it/s]


In [48]:
import faiss
# Create a Faiss index
index = faiss.IndexFlatIP(128)
# Add some vectors to the index
index.add(train_embeddings.detach().cpu().numpy())

In [49]:
pred_labels = [] 
for embedding,label in zip(test_embeddings.detach().cpu().numpy(),test_labels):
    _, indices = index.search(embedding.reshape(1,-1).astype(np.float32), 1)
    pred_class = train_labels[indices[0][0]]
    pred_labels.append(pred_class)

In [50]:
pred_labels = [i.detach().cpu().numpy() for i in pred_labels]

In [51]:
from sklearn.metrics import classification_report
report = classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys()),output_dict=True)

In [52]:
print(classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys())))

                                    precision    recall  f1-score   support

        001.Black_footed_Albatross       0.62      0.83      0.71        12
              002.Laysan_Albatross       0.90      0.75      0.82        12
               003.Sooty_Albatross       0.82      0.75      0.78        12
             004.Groove_billed_Ani       0.71      1.00      0.83        12
                005.Crested_Auklet       0.89      0.89      0.89         9
                  006.Least_Auklet       0.78      0.88      0.82         8
               007.Parakeet_Auklet       0.75      0.90      0.82        10
             008.Rhinoceros_Auklet       0.70      0.78      0.74         9
              009.Brewer_Blackbird       0.40      0.33      0.36        12
          010.Red_winged_Blackbird       1.00      1.00      1.00        12
               011.Rusty_Blackbird       0.54      0.58      0.56        12
       012.Yellow_headed_Blackbird       1.00      1.00      1.00        11
           

In [53]:
df = pd.DataFrame(report).transpose()

In [54]:
df.head()

,precision,recall,f1-score,support
001.Black_footed_Albatross,0.625000,0.833333,0.714286,12.0
002.Laysan_Albatross,0.900000,0.750000,0.818182,12.0
003.Sooty_Albatross,0.818182,0.750000,0.782609,12.0
004.Groove_billed_Ani,0.705882,1.000000,0.827586,12.0
005.Crested_Auklet,0.888889,0.888889,0.888889,9.0


In [55]:
df.reset_index(inplace=True)

In [56]:
df.rename(columns={"index":'class_name'},inplace=True)

In [57]:
classification_report_table = wandb.Table(dataframe=df)

In [58]:
run.log({"all_classes_classification_report": classification_report_table})

## Removing classes from training dataset and seeing performance

In [59]:
train_image_paths = []
train_labels = []
for folder_path,i in class_dict.items():
    if i >= 180:
        break
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    train_image_paths.extend(folder_images)
    train_labels.extend([i]*len(folder_images))

In [60]:
test_image_paths = []
test_labels = []
for folder_path,i in class_dict.items():
    if i >= 180:
        folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
        test_image_paths.extend(folder_images)
        test_labels.extend([i]*len(folder_images))

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_image_paths,train_labels, test_size=config['train_test_split'],
                                                    stratify=train_labels, random_state=config['random_seed'])

In [62]:
train_dataset  = CUBDataset(X_train,y_train,train_transform)
test_dataset  = CUBDataset(X_test,y_test,test_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

In [63]:
model = ResNetFeatrueExtractor50(pretrained=True)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'])
num_epochs = config['num_epochs']

### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.MeanReducer()
loss_func = losses.NTXentLoss(temperature=config['temperature'], distance=distance, reducer=reducer)
mining_func = miners.BatchEasyHardMiner(pos_strategy="easy",neg_strategy="semihard")
knn_func = CustomKNN(CosineSimilarity())
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",),k=1,knn_func=knn_func,avg_of_avgs=True,return_per_class=False)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [64]:
#total_loss = []
#total_acc = []
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, loss_func,mining_func, device, train_loader, optimizer, epoch)
    #total_loss.extend(train_loss)
    test_acc = test(train_dataset, test_dataset, model, accuracy_calculator)
    #total_acc.append(test_acc)
    wandb.log({"test_accuracy2": test_acc, "train_loss2": train_loss,"epoch": epoch})

Epoch 1 Iteration 0: Loss = 0.655242383480072


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:09<00:00,  6.86it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.39314814814814814
Epoch 2 Iteration 0: Loss = 0.5803188681602478


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:04<00:00, 14.36it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.4313173400673401
Epoch 3 Iteration 0: Loss = 0.5041823983192444


...

Computing accuracy
Test set accuracy (Precision@1) = 0.7581242985409653
Epoch 99 Iteration 0: Loss = 0.2269357591867447


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:04<00:00, 14.88it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.7576332772166107
Epoch 100 Iteration 0: Loss = 0.1929604411125183


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:04<00:00, 14.72it/s]

Computing accuracy
Test set accuracy (Precision@1) = 0.7631299102132437


In [65]:
torch.save(model,'models/cub_triplet_loss_epshn_resnet50_sgd_aug_180.pth')

In [32]:
model = torch.load('models/cub_triplet_loss_epshn_resnet18_sgd_aug_180.pth',map_location='cuda')

In [66]:
model.eval()

ResNetFeatrueExtractor50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequent

In [67]:
image_paths = []
labels = []
for folder_path,i in class_dict.items():
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    image_paths.extend(folder_images)
    labels.extend([i]*len(folder_images))

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=config['train_test_split'],
                                                    stratify=labels,
                                                    random_state=config['random_seed'])

In [69]:
train_dataset  = CUBDataset(X_train,y_train,transform)
test_dataset  = CUBDataset(X_test,y_test,transform)

In [70]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:11<00:00,  6.63it/s]


In [71]:
acc_dict2 = {}
for i in test_labels.unique():
    new_labels = test_labels[test_labels==i]
    new_embeddings = test_embeddings[test_labels==i]
    accuracies = accuracy_calculator.get_accuracy(
        new_embeddings, new_labels, train_embeddings, train_labels, False
    )
    acc_dict2[data_dict[int(i.detach().cpu().numpy())]]=[ 
                  len(new_labels),
                  len(train_labels[train_labels==i]),
                  accuracies["precision_at_1"]]
    print("{:<30} test samples {:<5}, training samples {:<5}: {}".format(data_dict[int(i.detach().cpu().numpy())],
                                                                  len(new_labels),
                                                                  len(train_labels[train_labels==i]),
                                                                  accuracies["precision_at_1"]))

001.Black_footed_Albatross     test samples 12   , training samples 48   : 0.8333333333333334
002.Laysan_Albatross           test samples 12   , training samples 48   : 0.8333333333333334
003.Sooty_Albatross            test samples 12   , training samples 46   : 0.9166666666666666
004.Groove_billed_Ani          test samples 12   , training samples 48   : 1.0
005.Crested_Auklet             test samples 9    , training samples 35   : 0.6666666666666666
006.Least_Auklet               test samples 8    , training samples 33   : 0.75
007.Parakeet_Auklet            test samples 10   , training samples 43   : 1.0
008.Rhinoceros_Auklet          test samples 9    , training samples 39   : 1.0
009.Brewer_Blackbird           test samples 12   , training samples 47   : 0.4166666666666667
010.Red_winged_Blackbird       test samples 12   , training samples 48   : 0.8333333333333334
011.Rusty_Blackbird            test samples 12   , training samples 48   : 0.6666666666666666
012.Yellow_headed_Blackbi

In [72]:
all_table = [[k]+v for k,v in acc_dict2.items()]
columns = ["class_name", "no. of test samples", "no. of train samples", "precision@1"]
train_table = wandb.Table(data=all_table, columns=columns)
run.log({"limited_classes_metrics": train_table})

In [73]:
import faiss
# Create a Faiss index
index = faiss.IndexFlatIP(128)
# Add some vectors to the index
index.add(train_embeddings.detach().cpu().numpy())

In [74]:
pred_labels = [] 
for embedding,label in zip(test_embeddings.detach().cpu().numpy(),test_labels):
    distances, indices = index.search(embedding.reshape(1,-1).astype(np.float32), 2)
    pred_class = train_labels[indices[0][1]]
    pred_labels.append(pred_class)

In [75]:
pred_labels = [i.detach().cpu().numpy() for i in pred_labels]

In [76]:
from sklearn.metrics import classification_report
print(classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys())))

                                    precision    recall  f1-score   support

        001.Black_footed_Albatross       0.50      0.75      0.60        12
              002.Laysan_Albatross       0.69      0.75      0.72        12
               003.Sooty_Albatross       0.75      0.75      0.75        12
             004.Groove_billed_Ani       0.85      0.92      0.88        12
                005.Crested_Auklet       0.89      0.89      0.89         9
                  006.Least_Auklet       1.00      0.75      0.86         8
               007.Parakeet_Auklet       0.80      0.80      0.80        10
             008.Rhinoceros_Auklet       0.78      0.78      0.78         9
              009.Brewer_Blackbird       0.25      0.25      0.25        12
          010.Red_winged_Blackbird       0.85      0.92      0.88        12
               011.Rusty_Blackbird       0.60      0.50      0.55        12
       012.Yellow_headed_Blackbird       1.00      1.00      1.00        11
           

In [77]:
from sklearn.metrics import classification_report
report = classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys()),output_dict=True)
df = pd.DataFrame(report).transpose()
df.reset_index(inplace=True)
df.rename(columns={"index":'class_name'},inplace=True)
classification_report_table = wandb.Table(dataframe=df)

In [78]:
run.log({"limited_classes_classification_report": classification_report_table})

## metric drop for unseen classes

In [79]:
unseen_class_names = sorted(key for key in acc_dict.keys() if int(key.split('.')[0]) >= 180)

In [82]:
comparison_table_data = []
for class_name in unseen_class_names:
    comparison_table_data.append([class_name,acc_dict[class_name][-1],acc_dict2[class_name][-1]])

In [83]:
comparison_table_data

[['180.Wilson_Warbler', 0.8333333333333334, 0.75],
 ['181.Worm_eating_Warbler', 0.9166666666666666, 0.5833333333333334],
 ['182.Yellow_Warbler', 0.6666666666666666, 0.5833333333333334],
 ['183.Northern_Waterthrush', 0.5, 0.3333333333333333],
 ['184.Louisiana_Waterthrush', 0.5833333333333334, 0.25],
 ['185.Bohemian_Waxwing', 0.9166666666666666, 0.8333333333333334],
 ['186.Cedar_Waxwing', 1.0, 0.8333333333333334],
 ['187.American_Three_toed_Woodpecker', 0.8, 0.8],
 ['188.Pileated_Woodpecker', 1.0, 0.6666666666666666],
 ['189.Red_bellied_Woodpecker', 1.0, 1.0],
 ['190.Red_cockaded_Woodpecker', 0.8181818181818182, 0.9090909090909091],
 ['191.Red_headed_Woodpecker', 0.8333333333333334, 0.8333333333333334],
 ['192.Downy_Woodpecker', 1.0, 1.0],
 ['193.Bewick_Wren', 0.6666666666666666, 0.75],
 ['194.Cactus_Wren', 0.8333333333333334, 1.0],
 ['195.Carolina_Wren', 0.6666666666666666, 0.5],
 ['196.House_Wren', 0.75, 0.4166666666666667],
 ['197.Marsh_Wren', 0.75, 0.5833333333333334],
 ['198.Rock_Wr

In [84]:
comparison_table = wandb.Table(data=comparison_table_data, columns=['class_name','all_classes_precision@1','unseen_classes_precision@1'])

In [85]:
run.log({"comparison_unseen_classes_metrics": comparison_table})

In [86]:
precisions = np.array(comparison_table_data)[:,1:3].astype(np.float32)

In [87]:
wandb.log({'precision_drop_unseen_classes':np.mean(np.subtract(precisions[:,0],precisions[:,1]))})

## metric drop for all classes because of new unseen classes

In [88]:
comparison_table_data = []
for class_name in acc_dict.keys():
    comparison_table_data.append([class_name,acc_dict[class_name][-1],acc_dict2[class_name][-1]])

In [89]:
precisions = np.array(comparison_table_data)[:,1:3].astype(np.float32)

In [90]:
precisions

array([[0.9166667 , 0.8333333 ],
       [0.6666667 , 0.8333333 ],
       [0.8333333 , 0.9166667 ],
       [1.        , 1.        ],
       [0.7777778 , 0.6666667 ],
       [0.875     , 0.75      ],
       [1.        , 1.        ],
       [0.6666667 , 1.        ],
       [0.41666666, 0.41666666],
       [1.        , 0.8333333 ],
       [0.5833333 , 0.6666667 ],
       [1.        , 0.90909094],
       [0.8333333 , 0.8333333 ],
       [0.8333333 , 0.8333333 ],
       [0.8181818 , 0.72727275],
       [0.72727275, 0.72727275],
       [0.90909094, 0.90909094],
       [1.        , 1.        ],
       [0.9166667 , 0.9166667 ],
       [0.8333333 , 0.8333333 ],
       [0.9166667 , 1.        ],
       [0.8181818 , 0.72727275],
       [0.41666666, 0.5       ],
       [1.        , 0.8       ],
       [0.6666667 , 0.5833333 ],
       [0.75      , 0.6666667 ],
       [0.33333334, 0.33333334],
       [1.        , 0.8333333 ],
       [0.41666666, 0.33333334],
       [0.08333334, 0.16666667],
       [0.

In [91]:
wandb.log({'precision_drop_all_classes':np.mean(np.subtract(precisions[:,0],precisions[:,1]))})